# Data Processing for Tabular Modeling
This notebook provides a **step-by-step walkthrough** of the data preprocessing pipeline. The goal is to prepare the dataset for machine learning by cleaning, feature engineering, and saving the processed data.

### Overview of Steps:
1. Import required libraries
2. Load raw data
3. Perform exploratory checks (shape, preview, missing values)
4. Handle missing values and clean data
5. Feature engineering
6. Save processed dataset


# Libraries

In [ ]:
import ee, geemap
import geopandas as gpd
from shapely.geometry import Point
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
import joblib
from os.path import join
from joblib import Parallel, delayed


# ee.Authenticate(force=True)
# ee.Initialize(project="your-project-id")

# Download Satellite Embeddings

In [ ]:
dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL');
aoi = ee.Geometry.BBox(north=-13.450000, south=-23.929167, west=-61.20833, east=-52.204167);
image = dataset.filterDate('2024-01-01', '2025-01-01').filterBounds(aoi).mosaic()

geemap.ee_export_image_to_drive(
    image=image,
    description=f'Embedding',
    folder='IGUIDE',
    region=aoi,
    dimensions="2161x2515",
    maxPixels=1e10
)

# Estimate Burn History

In [ ]:
burn_area = xr.open_dataset("/anvil/projects/x-cis250634/team2/Final_Data/Burn area/MCD64A1.061_500m_aid0001_202405_202407.nc", engine='rasterio',decode_times=True)
# hold time-indexed burn layers
burn_qc_list = []

for t in range(burn_area.sizes["time"]):
    burn_doy = burn_area["Burn_Date"][t, :, :]
    uncertainty = burn_area["Burn_Date_Uncertainty"][t, :, :]
    qa = burn_area["QA"][t, :, :]

    # Decode QA
    burned = burn_doy > 0
    good_uncertainty = uncertainty <= 5.0
    valid_data = (qa.astype(int) & (1 << 1)) > 0
    special_condition = (((qa.astype(int).values >> 5) & 0b111) == 0)
    qc_mask = burned & good_uncertainty & valid_data & special_condition

    # Mask the DOY values
    burn_doy_qc_masked = burn_doy.where(qc_mask, np.nan)

    # Create a DataArray with lat/lon dimensions
    burn_doy_qc = xr.DataArray(
        burn_doy_qc_masked.values,
        coords={'y': burn_doy_qc_masked.y, 'x': burn_doy_qc_masked.x},
        dims=['y', 'x']
    )

    # Add time coordinate for stacking
    burn_doy_qc = burn_doy_qc.expand_dims(dim={"time": [burn_area["time"][t].values]})

    burn_qc_list.append(burn_doy_qc)

# Stack along time dimension
burn_qc_all = xr.concat(burn_qc_list, dim="time")
burn_qc_all.name = "Burn_DOY_QC"

# Write CRS to the final stacked DataArray
burn_qc_all.rio.write_crs("EPSG:4326", inplace=True)

In [ ]:
# doys = range(121, 213)
lags = range(0, 6)
patch = 5

burn_roll_list = []
for i in range(burn_area.sizes["time"]):
    burn_doy = burn_qc_all.sel(time=i)
    doys = np.unique(burn_doy.values)
    for doy in tqdm(doys):
        for t in lags:
            burn_roll = (burn_doy==(doy-t)).astype(int).rolling(dim={"x":patch,"y":patch}, center=True).sum()
            burn_roll = burn_roll.expand_dims({"doy": 1, "lag": 1}).assign_coords({"doy": [doy], "lag": [t]})
            burn_roll_list.append(burn_roll)

burn_roll_all = xr.combine_by_coords(burn_roll_list)

joblib.dump(burn_roll_all, f"/anvil/projects/x-cis250634/team2/Final_Data/Rolling_Data/Burn_Roll.lzma")
burn_roll = joblib.load(f"/anvil/projects/x-cis250634/team2/Final_Data/Rolling_Data/Burn_Roll.lzma")
burn_roll

In [ ]:
def processBurnRoll(doy):
    da_doy = burn_roll.sel(doy=doy)
    da_doy = da_doy.transpose('y', 'x', 'lag')
    df = da_doy.to_dataframe().reset_index()
    df_pivot = df.pivot_table(index=["x", "y"], columns='lag', values='Burn_DOY_QC', dropna=False, fill_value=100)
    df_pivot.columns =[f'lag_{int(c)}' for c in df_pivot.columns]
    df_pivot.index.names = ["lon", "lat"]
    df_sorted = df_pivot.sort_index().astype("uint8")
    df_sorted.to_parquet(join(out_path, f"burn_roll_{doy}.parquet"), compression="gzip")

for doy in tqdm(burn_roll["doy"].values): processBurnRoll(doy)

# Process Raw Data

In [ ]:
embedding = rxr.open_rasterio(r"/anvil/projects/x-cis250634/team2/Final_Data/Embedding/Embedding.tif")

mean_da = embedding.mean(dim='band', keep_attrs=True).expand_dims(band=[65])
min_da = embedding.min(dim='band', keep_attrs=True).expand_dims(band=[66])
max_da = embedding.max(dim='band', keep_attrs=True).expand_dims(band=[67])
std_da = embedding.std(dim='band', keep_attrs=True).expand_dims(band=[68])
embedding_extended = xr.concat([embedding, mean_da, min_da, max_da, std_da], dim='band')

ds = embedding_extended.to_dataset(name='band_values')
ds = ds.transpose('y', 'x', 'band')
df = ds.to_dataframe().reset_index()
df = df.pivot(index=['y', 'x'], columns='band', values='band_values').reset_index()
df.columns.name = None
df = df.rename(columns={'x': 'lon', 'y': 'lat'})
df.columns = ['lat', 'lon'] + [f'band_{i}' for i in range(68)]

df = df.rename(columns={"band_64": "embed_mean", "band_65": "embed_min", "band_66": "embed_max", "band_67": "embed_std"})
df = df.set_index(["lon", "lat"]).sort_index().astype("float32")
df.to_parquet(f"/anvil/projects/x-cis250634/team2/Final_Data/Embedding/embedding.parquet", compression="gzip")

In [ ]:
file_path = r"/anvil/projects/x-cis250634/team2/Final_Data/CSV/*.csv"
csv_files = glob(file_path)
cols = ['grid_id', 'lon', 'lat', 'land_cover', 'dem', 't2m', 'd2m', 'u10', 'v10', 'tp', 'swvl1', 'sp', 'burn']

idx_cols = ['lon', 'lat']
int32_cols = ["grid_id"]
int8_cols = ["land_cover", "burn"]
float32_cols = ['dem', 't2m', 'd2m', 'u10', 'v10', 'tp', 'swvl1', 'sp']

out_path = r"D:\Projects\IGUIDE\Datasets\Raw_Data"

doys = pd.Series(csv_files)
doys = doys.apply(lambda p: datetime.strptime(os.path.splitext(os.path.basename(p))[0], "%Y-%m-%d").timetuple().tm_yday).values

def processRawData(path, doy):
    df = pd.read_csv(path, usecols=cols)
    df = pd.concat(    
        [   df[idx_cols],
            df[int32_cols].astype("uint32", errors="ignore"),
            df[int8_cols].astype("uint8", errors="ignore"),
            df[float32_cols].astype("float32", errors="ignore")],
        axis=1)

    df = df.set_index(["lon", "lat"]).sort_index()
    df.to_parquet(join(out_path, f"org_data_{doy-1}.parquet"), compression="gzip")

for path, doy in tqdm(zip(csv_files[33:], doys[33:])): processRawData(path, doy)

# Prepare Training Data

In [ ]:
main_dir = r"/anvil/projects/x-cis250634/team2/Final_Data"
raw_dir = join(main_dir, "Raw_Data")
rol_dir = join(main_dir, "Rolling_Data")
emb_dir = join(main_dir, "Embedding")
daily_dir = join(main_dir, "Data", "Daily")
train_dir = join(main_dir, "Data", "Training")

In [ ]:
max_burn = gpd.read_file(join(main_dir, "MaxBurnedArea.zip"))
raw_data = pd.read_parquet(join(raw_dir, "org_data_121.parquet"))

poi_data = raw_data.reset_index()[["lat", "lon", "grid_id"]]
poi_data = poi_data.sort_values(["lat", "lon",], ascending=[False, True])
poi_data = gpd.GeoDataFrame(
    data=poi_data,
    geometry=gpd.points_from_xy(poi_data['lon'], poi_data['lat']),
    crs='EPSG:4326'
)

filtered_points = poi_data[poi_data.within(max_burn.union_all())]
filtered_points.to_parquet(join(main_dir, "Points.parquet"))

In [ ]:
embedding_data = (
    pd.read_parquet(join(emb_dir, "embedding.parquet"))
    .reset_index()
    .sort_values(["lat", "lon"], ascending=[False, True])
    .reset_index(drop=True)
    .rename_axis('grid_id')
    .drop(columns=["lat", "lon"])
)

clipped_points = pd.read_parquet(join(main_dir, "Points.parquet"))["grid_id"]

In [ ]:
def combineData(doy):
    raw_data = (
        pd.read_parquet(join(raw_dir, f"org_data_{doy}.parquet"))
        .reset_index()
        .sort_values(["lat", "lon"], ascending=[False, True])
        .assign(doy=doy)
        .set_index(["doy", "grid_id"])
        
    )
    
    roll_data = (
        pd.read_parquet(join(rol_dir, f"burn_roll_{doy}.parquet"))
        .reset_index()
        .sort_values(["lat", "lon"], ascending=[False, True])
        .reset_index(drop=True)
        .rename_axis('grid_id')
        .assign(doy=doy)
        .set_index("doy", append=True)
        .reorder_levels(["doy", "grid_id"])
        .drop(columns=["lat", "lon"])
        
    )
    
    emb_data = (
        embedding_data
        .assign(doy=doy)
        .set_index("doy", append=True)
        .reorder_levels(["doy", "grid_id"])
    )
    
    
    data = pd.concat([raw_data, roll_data, emb_data], axis=1)
    data.to_parquet(join(daily_dir, f"data_{doy}.parquet"), compression="gzip")
    

def clipData(doy):
    df = (
        pd.read_parquet(join(daily_dir, f"data_{doy}.parquet"))
        .replace(dict(zip(lags, [100]*len(lags))), np.nan)
        .dropna(how="any")
    )
    
    df_whole = df.copy()
    df.index.get_level_values("grid_id")
    df_clipped = df[df.index.get_level_values("grid_id").isin(clipped_points)]
    return df_whole, df_clipped

def splitSample(df):
    burn_subset = df[df[y_column] == 1]
    nonburn_subset = df[df[y_column] == 0]

    if not (burn_subset.empty or nonburn_subset.empty):   
        n_burn = min(len(burn_subset), max_burn_per_day)
        sampled_burns = burn_subset.sample(n=n_burn, random_state=42)
        n_nonburn = min(len(nonburn_subset), nonburn_ratio * n_burn)
        sampled_nonburns = nonburn_subset.sample(n=n_nonburn, random_state=42)
        sample_df = pd.concat([sampled_burns, sampled_nonburns], axis=0)
    return sample_df

In [ ]:
y_column = 'burn'
max_burn_per_day = 5000
nonburn_ratio = 1
lags = ['lag_0', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

In [ ]:
samples_clip = []
samples_whole = []

for doy in  tqdm(range(121, 213)):
    df_whole, df_clip = clipData(doy)
    sample_whole = splitSample(df_whole)
    sample_clip = splitSample(df_clip)
    samples_whole.append(sample_whole)
    samples_clip.append(sample_clip)

In [ ]:
training_whole = pd.concat(samples_whole, axis=0).drop(columns={"lag_0"})
training_clip = pd.concat(samples_clip, axis=0).drop(columns={"lag_0"})

training_whole.to_parquet(join(train_dir, "training_whole.parquet"), compression="gzip")
training_clip.to_parquet(join(train_dir, "training_clip.parquet"), compression="gzip")